In [2]:
#Import important libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from scipy import stats
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')


from scipy import stats
from scipy.stats import norm, skew

In [3]:
#importing the data into dataframes
train_data = pd.read_csv('train_house.csv',low_memory=False)
test_data = pd.read_csv('test_house.csv',low_memory=False)

In [4]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [27]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

## Data Exploration

In [5]:
#
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
#col
sales=[],
vee=[]
for col in train_data.columns:
    vee = train_data[col].value_counts()

print(vee.head(),col)

140000    20
135000    17
155000    14
145000    14
190000    13
Name: SalePrice, dtype: int64 SalePrice


In [7]:
#the unique values from each columns
for col in train_data.columns:
    value = train_data[col].nunique()
    print(col,value)

Id 1460
MSSubClass 15
MSZoning 5
LotFrontage 110
LotArea 1073
Street 2
Alley 2
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
LandSlope 3
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
OverallQual 10
OverallCond 9
YearBuilt 112
YearRemodAdd 61
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 4
MasVnrArea 327
ExterQual 4
ExterCond 5
Foundation 6
BsmtQual 4
BsmtCond 4
BsmtExposure 4
BsmtFinType1 6
BsmtFinSF1 637
BsmtFinType2 6
BsmtFinSF2 144
BsmtUnfSF 780
TotalBsmtSF 721
Heating 6
HeatingQC 5
CentralAir 2
Electrical 5
1stFlrSF 753
2ndFlrSF 417
LowQualFinSF 24
GrLivArea 861
BsmtFullBath 4
BsmtHalfBath 3
FullBath 4
HalfBath 3
BedroomAbvGr 8
KitchenAbvGr 4
KitchenQual 4
TotRmsAbvGrd 12
Functional 7
Fireplaces 4
FireplaceQu 5
GarageType 6
GarageYrBlt 97
GarageFinish 3
GarageCars 5
GarageArea 441
GarageQual 5
GarageCond 5
PavedDrive 3
WoodDeckSF 274
OpenPorchSF 202
EnclosedPorch 120
3SsnPorch 20
ScreenPorch 76
PoolArea 8
PoolQC 3
Fence 4
MiscFeature 4
MiscV

In [8]:
mid_unique_values =[]

for col in train_data.columns:
    value = train_data[col].nunique()
    if (value > 20) and (value < 50):
        mid_unique_values.append(col)
print(mid_unique_values)

['Neighborhood', 'LowQualFinSF', 'MiscVal']


In [9]:
large_unique_values = []
for col in train_data.columns:
    value = train_data[col].nunique()
    if value > 50:
        large_unique_values.append(col)
print(large_unique_values)

['Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'SalePrice']


In [10]:
Neighborhood = train_data["Neighborhood"].value_counts().head(10)
Neighborhood

NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
Name: Neighborhood, dtype: int64

In [11]:
LowQualFinSF = train_data["LowQualFinSF"].value_counts().head(10)
LowQualFinSF

0      1434
80        3
360       2
392       1
528       1
515       1
514       1
513       1
481       1
479       1
Name: LowQualFinSF, dtype: int64

In [12]:
MiscVal = train_data["MiscVal"].value_counts().head(10)
MiscVal

0       1408
400       11
500        8
700        5
450        4
600        4
2000       4
480        2
1200       2
2500       1
Name: MiscVal, dtype: int64

In [21]:
def wrangle(datapath):
    #read data into dataframe
    df = pd.read_csv(datapath)
    #get list of features with above 20% missing values
    mask = df.isna().sum()[df.isna().sum()/len(df) > 0.2].keys().tolist()
    #drop the features above
    df.drop(columns=mask,inplace=True)
    

    
    #group unique feature values in 3 classes (multicollinerity columns)
    mid_unique_values =[]
    large_unique_values =[]
    target = []
    #concatinating features with mid and low counts of unique values
    for col in df.columns:
        value = df[col].nunique()
        if value < 25:
                mid_unique_values.append(col)
        elif col == "SalePrice":
            target.append(col)
        else:
                large_unique_values.append(col)
    data = pd.concat([df[mid_unique_values],df[target]],axis=1)
    return data

In [22]:
train = wrangle('train_house.csv')
test = wrangle('test_house.csv')

In [23]:
train.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,PavedDrive,PoolArea,MoSold,YrSold,SaleType,SaleCondition,Neighborhood,LowQualFinSF,MiscVal,SalePrice
0,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Y,0,2,2008,WD,Normal,CollgCr,0,0,208500
1,20,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,...,Y,0,5,2007,WD,Normal,Veenker,0,0,181500
2,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Y,0,9,2008,WD,Normal,CollgCr,0,0,223500
3,70,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,Y,0,2,2006,WD,Abnorml,Crawfor,0,0,140000
4,60,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,...,Y,0,12,2008,WD,Normal,NoRidge,0,0,250000


In [24]:
test.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageCond,PavedDrive,3SsnPorch,PoolArea,MoSold,YrSold,SaleType,SaleCondition,Neighborhood,MiscVal
0,20,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,...,TA,Y,0,0,6,2010,WD,Normal,NAmes,0
1,20,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,TA,Y,0,0,6,2010,WD,Normal,NAmes,12500
2,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,TA,Y,0,0,3,2010,WD,Normal,Gilbert,0
3,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,TA,Y,0,0,6,2010,WD,Normal,Gilbert,0
4,120,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,TA,Y,0,0,1,2010,WD,Normal,StoneBr,0


In [25]:
print(train.shape)
print(train.info())
print(train.columns.nunique())

(1460, 56)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1460 non-null   int64 
 1   MSZoning       1460 non-null   object
 2   Street         1460 non-null   object
 3   LotShape       1460 non-null   object
 4   LandContour    1460 non-null   object
 5   Utilities      1460 non-null   object
 6   LotConfig      1460 non-null   object
 7   LandSlope      1460 non-null   object
 8   Condition1     1460 non-null   object
 9   Condition2     1460 non-null   object
 10  BldgType       1460 non-null   object
 11  HouseStyle     1460 non-null   object
 12  OverallQual    1460 non-null   int64 
 13  OverallCond    1460 non-null   int64 
 14  RoofStyle      1460 non-null   object
 15  RoofMatl       1460 non-null   object
 16  Exterior1st    1460 non-null   object
 17  Exterior2nd    1460 non-null   object
 18  MasVnrType     14

In [26]:
print(test.shape)
print(test.info())
print(test.columns.nunique())

(1459, 56)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   Street         1459 non-null   object 
 3   LotShape       1459 non-null   object 
 4   LandContour    1459 non-null   object 
 5   Utilities      1457 non-null   object 
 6   LotConfig      1459 non-null   object 
 7   LandSlope      1459 non-null   object 
 8   Condition1     1459 non-null   object 
 9   Condition2     1459 non-null   object 
 10  BldgType       1459 non-null   object 
 11  HouseStyle     1459 non-null   object 
 12  OverallQual    1459 non-null   int64  
 13  OverallCond    1459 non-null   int64  
 14  RoofStyle      1459 non-null   object 
 15  RoofMatl       1459 non-null   object 
 16  Exterior1st    1458 non-null   object 
 17  Exterior2nd    1458 non-null   object 
 1

In [30]:
target_col = "SalePrice"
y_train = train[target_col]
train = train.drop(columns=target_col)

test =dtypes: float64(3), int64(15), object(38)
train =dtypes: int64(17), object(38)

In [31]:
#drop features not common to both train and test datasets
save = []
for col in test.columns:
      if col not in train.columns:
            save.append(col)
test.drop(columns=save,inplace=True)            

In [32]:
#recasting float types in test as ints
y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [33]:
#drop leaky features


In [76]:
categorical_data = train.select_dtypes(include='object').copy()
#for feature in features:
count = [len(train[features].unique()) for features in categorical_data]
data_tuples = list(zip(categorical_data,count))
datas = pd.DataFrame(data_tuples, columns=['Features','Number of distinct values'])
datas

,Features,Number of distinct values
0,MSZoning,5
1,Street,2
2,LotShape,4
3,LandContour,4
4,Utilities,2
5,LotConfig,5
6,LandSlope,3
7,Condition1,6
8,Condition2,6
9,BldgType,5


In [77]:
int_data = train.select_dtypes(include='int64').copy()
#for feature in features:
count = [len(train[features].unique()) for features in int_data]
data_tuple = list(zip(int_data,count))
dat = pd.DataFrame(data_tuple, columns=['Features','Number of distinct values'])
dat

,Features,Number of distinct values
0,MSSubClass,15
1,OverallQual,10
2,OverallCond,9
3,BsmtFullBath,4
4,BsmtHalfBath,3
5,FullBath,4
6,HalfBath,3
7,BedroomAbvGr,8
8,KitchenAbvGr,4
9,TotRmsAbvGrd,12


In [73]:
#concatinating features with mid and low counts of unique values
large_unique_value =[]
for col in categorical_data.columns:
    value = len(categorical_data[col].unique())
    if value > 5:
        large_unique_value.append(col)
print(large_unique_value)

['Condition1', 'Condition2', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional', 'GarageType', 'GarageQual', 'GarageCond', 'SaleType', 'SaleCondition', 'Neighborhood']


In [98]:
for val in large_unique_value:
    top_5 = train[val].value_counts().head(5)
    train[val] = train[val].apply(lambda c: c if c in top_5 else "Others")
    print(train[val].value_counts())

Norm      1260
Feedr       81
Artery      48
Others      45
RRAn        26
Name: Condition1, dtype: int64
Norm      1445
Feedr        6
Others       5
RRNn         2
Artery       2
Name: Condition2, dtype: int64
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
Others     33
Name: HouseStyle, dtype: int64
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Others        2
Name: RoofStyle, dtype: int64
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Others        4
Name: RoofMatl, dtype: int64
VinylSd    515
Others     297
HdBoard    222
MetalSd    220
Wd Sdng    206
Name: Exterior1st, dtype: int64
VinylSd    504
Others     338
MetalSd    214
HdBoard    207
Wd Sdng    197
Name: Exterior2nd, dtype: int64
PConc     647
CBlock    634
BrkTil    146
Slab       24
Stone       6
Others      3
Name: Foundation, dtype: int64
Unf       430
GLQ       418
ALQ       220
BLQ       148
Rec       133
Others    111
Name: BsmtFinType1, dtype

In [92]:
for val in large_unique_value:
    top_5 = train[val].value_counts().head(5)
    print(top_5)

Norm      1260
Feedr       81
Artery      48
Others      45
RRAn        26
Name: Condition1, dtype: int64
Norm      1445
Feedr        6
Others       5
RRNn         2
Artery       2
Name: Condition2, dtype: int64
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
Name: HouseStyle, dtype: int64
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Name: RoofStyle, dtype: int64
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Others        4
Name: RoofMatl, dtype: int64
VinylSd    515
Others     297
HdBoard    222
MetalSd    220
Wd Sdng    206
Name: Exterior1st, dtype: int64
VinylSd    504
Others     338
MetalSd    214
HdBoard    207
Wd Sdng    197
Name: Exterior2nd, dtype: int64
PConc     647
CBlock    634
BrkTil    146
Slab       24
Stone       6
Name: Foundation, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
Name: BsmtFinType1, dtype: int64
Unf       1256
Others      71
Rec         54
LwQ         46
BLQ  

In [96]:
Neighborhood = train_data["Condition1"].value_counts()
Neighborhood

Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: Condition1, dtype: int64